# Get a video dataset to do better training
- higher resolution than used in research

### super 8 look https://www.youtube.com/watch?v=7Q5UZmwxIXo
- low focus (diffusion and blur)
- low contrast
- camera is more shakey on home movies
- high depth of field everything is sharp
- 4x3
- film grain
- color changes

### VCR look
- jitter on scan lines - see http://www.avisynth.nl/users/vcmohan/DeJitter/DeJitter.htm
- chroma issues https://forum.videohelp.com/threads/397928-Imrpove-old-video-recorded-by-a-bad-camera?s=0a1230911434e7442d05b6b6cee8e6d2

## use images of related material to train superres
- family photos to train for family videos


## other datasets
wget https://cv.snu.ac.kr/research/EDSR/Flickr2K.tar
tar -xvf Flickr2K.tar


 

See this vimeo downloader: https://github.com/r0oth3x49/vimeo

In [1]:
import vimeo_dl as vimeo
from time import sleep
from tqdm import tqdm

In [2]:
url = "https://vimeo.com/140816903"

In [20]:
video = vimeo.new(url)

In [21]:
video.title, video.duration

('Just For Men - Foolproof', '00:31')

In [6]:
best

normal:mp4@1162x720

In [8]:
!pwd

/media/SSD/superres


In [2]:
vid_list_fn = "/media/SSD/superres/pytorch-vdsr/data/original_video_list.txt"
path = "/media/SSD/superres/video_data/"

In [3]:
with open(vid_list_fn) as f:
    content = f.readlines()
content = [x.strip() for x in content] 

In [5]:
bad=0
rescount=292

In [6]:
for url in tqdm(content[601:1201]):
    try:
        video = vimeo.new(url)
        streams = video.streams
    except TypeError:
        #print("Bad URL:", url)
        bad=bad+1
    else:        
        
        for s in streams:
            if s.extension == 'mp4' and s.resolution == '1280x720' :
                
                filename = path + url[url.find('/',8)+1:]+".mp4"
                #print(url,s.resolution,"->",filename)
                s.download(filename,quiet=True)
                rescount=rescount+1
    
    #sleep(2)

100%|██████████| 600/600 [39:50<00:00,  3.98s/it]  


In [7]:
rescount

620

In [2]:
vid_list_fn = "/media/SSD/superres/list of vimeo movies"
path = "/media/SSD/superres/video_data/"

In [3]:
with open(vid_list_fn) as f:
    content = f.readlines()
content = [x.strip() for x in content] 

In [4]:
bad=0
rescount=620

In [5]:
for url in tqdm(content):
    try:
        video = vimeo.new(url)
        streams = video.streams
    except TypeError:
        #print("Bad URL:", url)
        bad=bad+1
    else:        
        
        for s in streams:
            if s.extension == 'mp4' and s.resolution == '1280x720' :
                
                filename = path + url[url.find('/',8)+1:]+".mp4"
                #print(url,s.resolution,"->",filename)
                s.download(filename,quiet=True)
                rescount=rescount+1
    
    #sleep(2)

 94%|█████████▍| 15/16 [02:36<00:10, 10.46s/it]


UnboundLocalError: local variable 'video_id' referenced before assignment

In [6]:
content

['https://vimeo.com/68445748',
 'https://vimeo.com/178858950',
 'https://vimeo.com/420951241',
 'https://vimeo.com/137712021',
 'https://vimeo.com/292184739',
 'https://vimeo.com/89190137',
 'https://vimeo.com/55172677',
 'https://vimeo.com/182932029',
 'https://vimeo.com/10450125',
 'https://vimeo.com/13181904',
 'https://vimeo.com/17851496',
 'https://vimeo.com/44060692',
 'https://vimeo.com/120061340',
 'https://vimeo.com/178500853',
 'https://vimeo.com/269305059',
 '']

# test the load for each image

In [ ]:
# preprocess video from wmv to mp4

In [ ]:
#!ffmpeg -i '/home/filip/Videos/1987 first video making muffins.wmv'  -c:v libx264 -crf 23 -c:a aac -strict -2 -q:a 100 '/media/SSD/superres/1987 first video making muffins.mp4'

In [ ]:
#!python /media/SSD/superres/Zooming-Slow-Mo-CVPR-2020/codes/video_to_zsm.py --video /media/SSD/superres/snip.mp4  --model /media/SSD/superres/model/xiang2020zooming.pth --output /media/SSD/superres/muffins_test.mp4


In [1]:
!pwd

/media/SSD/superres


To deinterlace
https://video.stackexchange.com/questions/17396/how-to-deinterlacing-with-ffmpeg
https://macilatthefront.blogspot.com/2017/04/deinterlacing-hd-footage-without-losing.html


In [1]:

#%reload_ext autoreload
#%autoreload 2
#%matplotlib inline


import cv2 
import os
import numpy as np
import subprocess as sp
import time
from tqdm import tqdm, trange
from pathlib import Path
import torch
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader


import sys
sys.path.insert(0, '/media/SSD/superres/pytorch-vdsr/')
#import vdsr
from vdsr import Net

In [2]:
torch.cuda.manual_seed(42)

Create a data loader
- see https://github.com/2KangHo/vdsr_pytorch/blob/master/data.py
- see https://github.com/2KangHo/vdsr_pytorch/blob/master/data_utils.py

In [3]:
import torch.utils.data as data

from os import listdir
from os.path import join
from PIL import Image, ImageFilter
import numpy as np
import torchvision as vision


toPIL = vision.transforms.ToPILImage()


def noisy(img, std=3.0):
    mean = 0.0
    gauss = np.random.normal(mean, std, (img.height, img.width, 3))
    # noisy = np.clip(np.uint8(img + gauss), 0, 255)
    noisy = np.uint8(img + gauss)
    return noisy


def is_image_file(filename):
    return any(filename.endswith(extension) for extension in [".png", ".jpg", ".jpeg"])


def load_img(filepath):
    img = Image.open(filepath).convert('YCbCr')
    return img


class DatasetFromFolder(data.Dataset):
    def __init__(self, image_dir, input_transform=None, target_transform=None, add_noise=None, noise_std=3.0):
        super(DatasetFromFolder, self).__init__()
        self.image_filenames = [join(image_dir, x)
                                for x in listdir(image_dir) if is_image_file(x)]

        self.input_transform = input_transform
        self.target_transform = target_transform
        self.add_noise = add_noise
        self.noise_std = noise_std

    def __getitem__(self, index):
        input = load_img(self.image_filenames[index])
        target = input.copy()
        if self.input_transform:
            if self.add_noise:
                input = noisy(input, self.noise_std)
                input = toPIL(input)
            input = self.input_transform(input)
        if self.target_transform:
            target = self.target_transform(target)

        return input, target

    def __len__(self):
        return len(self.image_filenames)

In [4]:
from torchvision.transforms import Compose, CenterCrop, ToTensor, Resize, Grayscale




def calculate_valid_crop_size(crop_size, upscale_factor):
    return crop_size - (crop_size % upscale_factor)


def input_transform(crop_size, upscale_factor):
    return Compose([
        Grayscale(num_output_channels=1),
        CenterCrop(crop_size),
        Resize((crop_size//upscale_factor, crop_size//upscale_factor)),
        Resize((crop_size, crop_size)),        
        ToTensor(),
    ])


def target_transform(crop_size):
    return Compose([
        Grayscale(num_output_channels=1),
        CenterCrop(crop_size),
        ToTensor(),
    ])


def get_training_set(data_dir, crop_size, upscale_factor, add_noise=None, noise_std=3.0):
    
    cropsize = calculate_valid_crop_size(crop_size, upscale_factor)

    return DatasetFromFolder(data_dir,
                             input_transform=input_transform(
                                 cropsize, upscale_factor),
                             target_transform=target_transform(cropsize),
                             add_noise=add_noise,
                             noise_std=noise_std)


def get_validation_set(data_dir, crop_size, upscale_factor):
    
    cropsize = calculate_valid_crop_size(crop_size, upscale_factor)

    return DatasetFromFolder(data_dir,
                             input_transform=input_transform(
                                 cropsize, upscale_factor),
                             target_transform=target_transform(cropsize))


def get_test_set(data_dir, crop_size, upscale_factor):

    cropsize = calculate_valid_crop_size(crop_size, upscale_factor)

    return DatasetFromFolder(data_dir,
                             input_transform=input_transform(
                                 cropsize, upscale_factor),
                             target_transform=target_transform(cropsize))

In [5]:
train_dir = "/media/SSD/fastai_data/pets/oxford-iiit-pet/images"

In [6]:
train_set = get_training_set(data_dir=train_dir, crop_size=256,
            upscale_factor=2,add_noise=False, noise_std=3.0)
training_data_loader = DataLoader(dataset=train_set, num_workers=0, batch_size=16, shuffle=True)

Build the model

In [7]:
#model = Net()
criterion = nn.MSELoss(size_average=False)

/media/SSD/conda_env/superres/lib/python3.8/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [8]:
model = torch.load("/media/SSD/superres/pytorch-vdsr/model/model_epoch_50.pth")["model"]

/media/SSD/conda_env/superres/lib/python3.8/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/media/SSD/conda_env/superres/lib/python3.8/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/media/SSD/conda_env/superres/lib/python3.8/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original sourc

In [9]:
init_lr = 0.1
lr_step = 5
gradient_clip = 0.4

In [10]:
optimizer = optim.SGD(model.parameters(), lr=init_lr, momentum=0.9, weight_decay=1e-4)

In [11]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 10 epochs"""
    lr = init_lr * (0.1 ** (epoch // lr_step))
    return lr

In [12]:
def save_checkpoint(model, epoch):
    model_out_path = "checkpoint/" + "model_epoch_{}.pth".format(epoch)
    state = {"epoch": epoch ,"model": model}
    if not os.path.exists("checkpoint/"):
        os.makedirs("checkpoint/")

    torch.save(state, model_out_path)

    print("Checkpoint saved to {}".format(model_out_path))

In [13]:
def train(training_data_loader, optimizer, model, criterion, epoch):
    lr = adjust_learning_rate(optimizer, epoch-1)

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

    print("Epoch = {}, lr = {}".format(epoch, optimizer.param_groups[0]["lr"]))

    model.train()

    for iteration, batch in enumerate(training_data_loader, 1):
        input, target = Variable(batch[0]), Variable(batch[1], requires_grad=False)

        
        input = input.cuda()
        target = target.cuda()

        loss = criterion(model(input), target)
        optimizer.zero_grad()
        loss.backward() 
        nn.utils.clip_grad_norm(model.parameters(),gradient_clip) 
        optimizer.step()

        if iteration%10 == 0:
            print("===> Epoch[{}]({}/{}): Loss: {:.10f}".format(epoch, iteration, len(training_data_loader), loss.data))

In [14]:
for epoch in range(1, 20):
        train(training_data_loader, optimizer, model, criterion, epoch)
        save_checkpoint(model, epoch)

Epoch = 1, lr = 0.1


<ipython-input-13-25f571245f6d>:21: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(model.parameters(),gradient_clip)


===> Epoch[1](10/462): Loss: 1332.9090576172
===> Epoch[1](20/462): Loss: 1218.9575195312
===> Epoch[1](30/462): Loss: 1339.7012939453
===> Epoch[1](40/462): Loss: 2574.2861328125
===> Epoch[1](50/462): Loss: 1563.3532714844
===> Epoch[1](60/462): Loss: 1126.9069824219
===> Epoch[1](70/462): Loss: 771.7797851562
===> Epoch[1](80/462): Loss: 911.9238281250
===> Epoch[1](90/462): Loss: 1487.5745849609
===> Epoch[1](100/462): Loss: 1496.0246582031
===> Epoch[1](110/462): Loss: 1133.2005615234
===> Epoch[1](120/462): Loss: 1124.4964599609
===> Epoch[1](130/462): Loss: 754.0661621094
===> Epoch[1](140/462): Loss: 1811.2097167969
===> Epoch[1](150/462): Loss: 2122.0517578125
===> Epoch[1](160/462): Loss: 1187.7091064453
===> Epoch[1](170/462): Loss: 764.3685913086
===> Epoch[1](180/462): Loss: 1292.7869873047
===> Epoch[1](190/462): Loss: 1626.5283203125
===> Epoch[1](200/462): Loss: 774.4918212891
===> Epoch[1](210/462): Loss: 1011.1511840820
===> Epoch[1](220/462): Loss: 1241.6042480469
==

===> Epoch[4](390/462): Loss: 1291.8695068359
===> Epoch[4](400/462): Loss: 1062.6291503906
===> Epoch[4](410/462): Loss: 1220.3979492188
===> Epoch[4](420/462): Loss: 1018.5102539062
===> Epoch[4](430/462): Loss: 993.8936767578
===> Epoch[4](440/462): Loss: 689.9306640625
===> Epoch[4](450/462): Loss: 1357.5982666016
===> Epoch[4](460/462): Loss: 1547.5722656250
Checkpoint saved to checkpoint/model_epoch_4.pth
Epoch = 5, lr = 0.1
===> Epoch[5](10/462): Loss: 796.9346923828
===> Epoch[5](20/462): Loss: 1856.7875976562
===> Epoch[5](30/462): Loss: 1430.0786132812
===> Epoch[5](40/462): Loss: 1565.1282958984
===> Epoch[5](50/462): Loss: 1074.0983886719
===> Epoch[5](60/462): Loss: 1573.1795654297
===> Epoch[5](70/462): Loss: 1021.5621337891
===> Epoch[5](80/462): Loss: 962.2158203125
===> Epoch[5](90/462): Loss: 937.5678100586
===> Epoch[5](100/462): Loss: 1196.1110839844
===> Epoch[5](110/462): Loss: 747.2858886719
===> Epoch[5](120/462): Loss: 916.1271972656
===> Epoch[5](130/462): Los

===> Epoch[8](280/462): Loss: 1455.4941406250
===> Epoch[8](290/462): Loss: 1929.8295898438
===> Epoch[8](300/462): Loss: 1075.2585449219
===> Epoch[8](310/462): Loss: 1024.4141845703
===> Epoch[8](320/462): Loss: 1283.0803222656
===> Epoch[8](330/462): Loss: 1183.7980957031
===> Epoch[8](340/462): Loss: 950.0357666016
===> Epoch[8](350/462): Loss: 784.3314208984
===> Epoch[8](360/462): Loss: 1040.4631347656
===> Epoch[8](370/462): Loss: 1343.9645996094
===> Epoch[8](380/462): Loss: 1081.0681152344
===> Epoch[8](390/462): Loss: 932.1453857422
===> Epoch[8](400/462): Loss: 992.7127685547
===> Epoch[8](410/462): Loss: 1325.1328125000
===> Epoch[8](420/462): Loss: 1095.1545410156
===> Epoch[8](430/462): Loss: 873.1282958984
===> Epoch[8](440/462): Loss: 849.8325195312
===> Epoch[8](450/462): Loss: 1305.6027832031
===> Epoch[8](460/462): Loss: 881.7731933594
Checkpoint saved to checkpoint/model_epoch_8.pth
Epoch = 9, lr = 0.010000000000000002
===> Epoch[9](10/462): Loss: 1008.2532958984
==

===> Epoch[12](150/462): Loss: 1046.8128662109
===> Epoch[12](160/462): Loss: 1050.8325195312
===> Epoch[12](170/462): Loss: 1324.5930175781
===> Epoch[12](180/462): Loss: 1219.5600585938
===> Epoch[12](190/462): Loss: 1323.1435546875
===> Epoch[12](200/462): Loss: 952.6604614258
===> Epoch[12](210/462): Loss: 807.1750488281
===> Epoch[12](220/462): Loss: 1543.2919921875
===> Epoch[12](230/462): Loss: 1285.3745117188
===> Epoch[12](240/462): Loss: 1066.1480712891
===> Epoch[12](250/462): Loss: 1536.1228027344
===> Epoch[12](260/462): Loss: 880.3024902344
===> Epoch[12](270/462): Loss: 729.1774902344
===> Epoch[12](280/462): Loss: 1476.2973632812
===> Epoch[12](290/462): Loss: 845.4057006836
===> Epoch[12](300/462): Loss: 1069.2635498047
===> Epoch[12](310/462): Loss: 591.9670410156
===> Epoch[12](320/462): Loss: 2100.3774414062
===> Epoch[12](330/462): Loss: 582.0338134766
===> Epoch[12](340/462): Loss: 847.1041259766
===> Epoch[12](350/462): Loss: 1111.8017578125
===> Epoch[12](360/46

===> Epoch[16](10/462): Loss: 1153.3150634766
===> Epoch[16](20/462): Loss: 1350.1535644531
===> Epoch[16](30/462): Loss: 1265.7330322266
===> Epoch[16](40/462): Loss: 1784.8914794922
===> Epoch[16](50/462): Loss: 998.7589721680
===> Epoch[16](60/462): Loss: 1107.1409912109
===> Epoch[16](70/462): Loss: 1134.6346435547
===> Epoch[16](80/462): Loss: 1194.3013916016
===> Epoch[16](90/462): Loss: 911.9483642578
===> Epoch[16](100/462): Loss: 1041.9838867188
===> Epoch[16](110/462): Loss: 1078.4066162109
===> Epoch[16](120/462): Loss: 1366.3137207031
===> Epoch[16](130/462): Loss: 1033.8145751953
===> Epoch[16](140/462): Loss: 1499.9614257812
===> Epoch[16](150/462): Loss: 969.0288696289
===> Epoch[16](160/462): Loss: 1448.9743652344
===> Epoch[16](170/462): Loss: 586.0979003906
===> Epoch[16](180/462): Loss: 690.6323242188
===> Epoch[16](190/462): Loss: 890.1898193359
===> Epoch[16](200/462): Loss: 759.0410766602
===> Epoch[16](210/462): Loss: 684.1329345703
===> Epoch[16](220/462): Loss:

===> Epoch[19](340/462): Loss: 1189.7473144531
===> Epoch[19](350/462): Loss: 991.3833007812
===> Epoch[19](360/462): Loss: 1510.6101074219
===> Epoch[19](370/462): Loss: 1469.0603027344
===> Epoch[19](380/462): Loss: 2487.9348144531
===> Epoch[19](390/462): Loss: 1596.5756835938
===> Epoch[19](400/462): Loss: 1601.6894531250
===> Epoch[19](410/462): Loss: 938.0840454102
===> Epoch[19](420/462): Loss: 1691.3320312500
===> Epoch[19](430/462): Loss: 1399.6654052734
===> Epoch[19](440/462): Loss: 1256.7355957031
===> Epoch[19](450/462): Loss: 931.3432617188
===> Epoch[19](460/462): Loss: 904.8264160156
Checkpoint saved to checkpoint/model_epoch_19.pth


In [10]:
print(model)

Net(
  (residual_layer): Sequential(
    (0): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (1): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (2): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (3): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (4): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (5): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (6): Conv_ReLU_Bl

Function that takes in a .mp4 video file and applies intrinsic matrix and distortion coefficients to undistort a video.  It also preserves the sound. it uses ffmpeg for some processing as well as opencv cv2 library.

In [15]:
# Define the colorization function
# We'll reuse the Cb and Cr channels from bicubic interpolation
def colorize(y, ycbcr): 
    img = np.zeros((y.shape[0], y.shape[1], 3), np.uint8)
    img[:,:,0] = y
    img[:,:,1] = ycbcr[:,:,1]
    img[:,:,2] = ycbcr[:,:,2]
    img = cv2.cvtColor(img, cv2.COLOR_YCrCb2BGR)
    return img

In [16]:
def superres_mp4(infile, outfile, model, factor=1.0,display=True,bitrate= "12000k",errorfile = None):
    
    model = model.cuda()
    #torch.set_grad_enabled(False)
    #model.eval()
    
    cap = cv2.VideoCapture(str(infile))
    
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)
    
    success_flag = False
    
    print("Original File:", str(infile))
    print("frames=",length,"\nwidth=",width,"\nheight=",height,"\nfps=",fps)
    
    

    new_height = int(height*factor)
    new_width = int(width*factor)
    
    print("\nProcess File:", str(outfile))
    print("factor:",factor,"\nwidth=",new_width, "\nheight=",new_height,"\nbitrate=",bitrate)
    
    

    dimension = '{}x{}'.format(new_width, new_height)  #ffmpeg uses bicubic as default scaling alg
    f_format = 'bgr24' # remember OpenCV uses bgr format




    command = ['ffmpeg',
            '-y',
            '-f', 'rawvideo',
            '-vcodec','rawvideo',
            '-s', dimension,
            '-pix_fmt', 'bgr24',
            '-r', str(fps),
            '-i', '-',
            '-i', str(infile),
            '-c:v', 'h264',
            '-c:a', 'aac',

            '-map','0:v:0',
            '-map','1:a:0',
            '-shortest',
            '-b:v', bitrate, 
            str(outfile) ]


    if errorfile is not None:
        ef = open(error_file,"w+")
        p = sp.Popen(command, stdin=sp.PIPE, stderr=ef)
    else:
        p = sp.Popen(command, stdin=sp.PIPE)

    # Full processing with a stream instead of a temp file for video
    pbar = tqdm(total=length)
    while(cap.isOpened()):




        # Capture frame-by-frame
        ret, frame = cap.read()
        if ret == True:
            
            if (factor != 1.0):
                frame = cv2.resize(frame, (new_width, new_height), interpolation=cv2.INTER_CUBIC)
            im_b_ycbcr = cv2.cvtColor(frame, cv2.COLOR_BGR2YCR_CB)
            im_b_y = im_b_ycbcr[:,:,0].astype(float)
            im_input = im_b_y/255.
            im_input = Variable(torch.from_numpy(im_input).float()).view(1, -1, im_input.shape[0], im_input.shape[1])
            im_input = im_input.cuda()
            out = model(im_input)

            out = out.cpu()
            im_h_y = out.data[0].numpy().astype(np.float32)
            im_h_y = im_h_y * 255.
            im_h_y[im_h_y < 0] = 0
            im_h_y[im_h_y > 255.] = 255.
            im_h_y = im_h_y[0,:,:]

            im_h = colorize(im_h_y, im_b_ycbcr)
            
                

            p.stdin.write(im_h.tobytes())


            if display:
                cv2.imshow('Processed',im_h)
                time.sleep(10)
                #cv2.imshow('Orig',frame)
            pbar.update(1)
            # Press Q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                success_flag = False
                break
        # Break the loop
        else:
            success_flag = True
            break
    # When everything done, release the video capture object

    pbar.close()

    p.stdin.close()
    p.wait()

    cap.release()

    # Closes all the frames
    cv2.destroyAllWindows()
    
    return success_flag

In [17]:
infile = "/media/SSD/superres/super8/super8_best_from_mp2.mp4"
outfile = "/media/SSD/superres/super8test2.mp4"
error_file = "/media/SSD/superres/error.txt"

In [6]:
# Load the pretrained model

model = torch.load("/media/SSD/superres/pytorch-vdsr/model/model_epoch_50.pth")["model"]

/media/SSD/conda_env/superres/lib/python3.8/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/media/SSD/conda_env/superres/lib/python3.8/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/media/SSD/conda_env/superres/lib/python3.8/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original sourc

In [18]:
superres_mp4(infile, outfile, model, factor=1.0,display=False,bitrate= "4000k",errorfile = error_file)

  0%|          | 1/7195 [00:00<16:24,  7.31it/s]

Original File: /media/SSD/superres/super8/super8_best_from_mp2.mp4
frames= 7195 
width= 720 
height= 480 
fps= 59.94005994005994

Process File: /media/SSD/superres/super8test2.mp4
factor: 1.0 
width= 720 
height= 480 
bitrate= 4000k


100%|██████████| 7195/7195 [10:59<00:00, 10.91it/s]


True

In [22]:
from multiprocessing.pool import ThreadPool
from collections import deque


def superres_parallel_mp4(infile, outfile, model, factor=1.0,display=True,bitrate= "12000k",errorfile = None):

    class DummyTask:
        def __init__(self, data):
            self.data = data
        def ready(self):
            return True
        def get(self):
            return self.data





    
    
    model = model.cpu()
    #torch.set_grad_enabled(False)
    #model.eval()
    
    cap = cv2.VideoCapture(str(infile))
    
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)
    
    success_flag = False
    
    print("Original File:", str(infile))
    print("frames=",length,"\nwidth=",width,"\nheight=",height,"\nfps=",fps)
    
    

    new_height = int(height*factor)
    new_width = int(width*factor)
    
    print("\nProcess File:", str(outfile))
    print("factor:",factor,"\nwidth=",new_width, "\nheight=",new_height,"\nbitrate=",bitrate)
    
    

    dimension = '{}x{}'.format(new_width, new_height)  #ffmpeg uses bicubic as default scaling alg
    f_format = 'bgr24' # remember OpenCV uses bgr format




    command = ['ffmpeg',
            '-y',
            '-f', 'rawvideo',
            '-vcodec','rawvideo',
            '-s', dimension,
            '-pix_fmt', 'bgr24',
            '-r', str(fps),
            '-i', '-',
            '-i', str(infile),
            '-c:v', 'h264',
            '-c:a', 'aac',
torc
            '-map','0:v:0',
            '-map','1:a:0',
            '-shortest',
            '-b:v', bitrate, 
            str(outfile) ]


    if errorfile is not None:
        ef = open(error_file,"w+")
        p = sp.Popen(command, stdin=sp.PIPE, stderr=ef)
    else:
        p = sp.Popen(command, stdin=sp.PIPE)   
    
    
 
    def process_frame(frame, t0):
        
        if (factor != 1.0):
            frame = cv2.resize(frame, (new_width, new_height), interpolation=cv2.INTER_CUBIC)
        im_b_ycbcr = cv2.cvtColor(frame, cv2.COLOR_BGR2YCR_CB)
        im_b_y = im_b_ycbcr[:,:,0].astype(float)
        im_input = im_b_y/255.
        im_input = Variable(torch.from_numpy(im_input).float()).view(1, -1, im_input.shape[0], im_input.shape[1])
        #im_input = im_input.cuda()
        out = model(im_input)

        #out = out.cpu()
        im_h_y = out.data[0].numpy().astype(np.float32)
        im_h_y = im_h_y * 255.
        im_h_y[im_h_y < 0] = 0
        im_h_y[im_h_y > 255.] = 255.
        im_h_y = im_h_y[0,:,:]

        im_h = colorize(im_h_y, im_b_ycbcr)        
        
        
        return im_h, t0

    
    
    
    threadn = cv2.getNumberOfCPUs()
    pool = ThreadPool(processes = threadn)
    pending = deque()

    threaded_mode = True

    
    while True:
        while len(pending) > 0 and pending[0].ready():
            res, t0 = pending.popleft().get()
            p.stdin.write(res.tobytes())
            
            
            cv2.imshow('threaded video', res)
        if len(pending) < threadn:
            _ret, frame = cap.read()
            t = 0
            
            last_frame_time = t
            if threaded_mode:
                task = pool.apply_async(process_frame, (frame.copy(), t))
            else:
                task = DummyTask(process_frame(frame, t))
            pending.append(task)
        ch = cv2.waitKey(1)
        if ch == ord(' '):
            threaded_mode = not threaded_mode
        if ch == 27:
            break

    print('Done')
    
    p.stdin.close()
    p.wait()

    cap.release()

    # Closes all the frames
    cv2.destroyAllWindows()



In [ ]:
superres_parallel_mp4(infile, outfile, model, factor=1.0,display=False,bitrate= "4000k",errorfile = error_file)

Original File: /media/SSD/superres/sup8snip.mp4
frames= 1797 
width= 720 
height= 480 
fps= 29.97002997002997

Process File: /media/SSD/superres/super8test.mp4
factor: 1.0 
width= 720 
height= 480 
bitrate= 4000k


In [17]:
model

Net(
  (residual_layer): Sequential(
    (0): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (1): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (2): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (3): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (4): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (5): Conv_ReLU_Block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
    )
    (6): Conv_ReLU_Bl

In [ ]:
infolder = Path("to_do_videos")
outfolder = Path("undistorted_videos")
completefolder = Path("raw_videos")

outfolder.mkdir(exist_ok=True)
completefolder.mkdir(exist_ok=True)


In [ ]:
for filename in infolder.iterdir():
    

    undistort_outfile = outfolder/("undistort"+filename.name)
    
    retval = undistort_mp4(infile=filename, outfile=undistort_outfile,
                           intrinsic_matrix=im, distortion_coefficients=dc,
                           crop=True,display=False,bitrate= "12000k",errorfile = None)   
    
    if retval:
        #move the source file if successful
        completefile = completefolder/filename.name
        filename.rename(completefile)